# Appendix - HW 1 Szymon Sadkowski

## Fetching the dataset and first analysis.

In [88]:
!pip install tabpfn

In [89]:
SEED = 42
TEST_SIZE = 0.2

In [90]:
from sklearn.datasets import fetch_openml

X, Y = fetch_openml('kc1', return_X_y=True)
X.head()
print(X.shape)

(2109, 21)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning:

The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.



In [91]:
import plotly.express as px
import numpy as np


px.histogram(Y, histnorm='probability density').show()
X.describe()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,...,2109.00000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000,2109.000000
mean,20.372262,2.838028,1.674443,2.546420,49.829445,258.696719,0.319583,6.771242,21.240071,5242.386240,...,291.24504,14.525367,0.945946,1.759602,0.132764,7.631674,9.537316,31.043717,18.786724,4.665908
std,29.754442,3.900763,2.200659,3.375859,83.599874,516.317605,0.317029,7.863646,21.500367,17444.981211,...,969.16516,24.188302,3.085271,3.856850,0.704023,5.730347,12.195727,51.776056,32.074398,7.792206
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.000000,1.000000,1.000000,1.000000,4.000000,8.000000,0.080000,1.500000,5.330000,12.000000,...,0.67000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,3.000000,1.000000,1.000000
50%,9.000000,1.000000,1.000000,1.000000,16.000000,57.060000,0.200000,3.500000,14.400000,213.970000,...,11.89000,5.000000,0.000000,0.000000,0.000000,6.000000,5.000000,10.000000,6.000000,1.000000
75%,24.000000,3.000000,1.000000,3.000000,58.000000,265.930000,0.670000,9.200000,29.850000,2276.020000,...,126.45000,17.000000,0.000000,2.000000,0.000000,11.000000,13.000000,36.000000,22.000000,5.000000
max,288.000000,45.000000,26.000000,45.000000,1106.000000,7918.820000,2.000000,53.750000,193.060000,324803.510000,...,18044.64000,262.000000,44.000000,58.000000,12.000000,37.000000,120.000000,678.000000,428.000000,89.000000


## Data split and testing models.

In [92]:
from tabpfn import TabPFNClassifier


class QuietTabPFNClassifier(TabPFNClassifier):
    def fit(self, *args, **kwargs):
        return super().fit(*args, **kwargs, overwrite_warning=True)


In [93]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder


__id = lambda x: x

models_dict = {
    'logistic_regression': LogisticRegression(random_state=SEED, max_iter=1000, n_jobs=-1),
    'random_forest': RandomForestClassifier(random_state=SEED, n_jobs=-1),
    'xgboost': XGBClassifier(random_state=SEED, n_jobs=-1),
    'tabpfn': QuietTabPFNClassifier(device='cpu', N_ensemble_configurations=2),
}

scaler_dict = {
    'tabpfn': FunctionTransformer(func=__id, inverse_func=__id),
    'logistic_regression': StandardScaler(),
    'random_forest': StandardScaler(),
    'xgboost': StandardScaler(),
}


Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [94]:
from typing import Tuple
import numpy as np


def prep_data(model_name, x, y) -> Tuple[np.ndarray, np.ndarray]:
    y_trans = LabelEncoder()
    y = y_trans.fit_transform(y)

    x_trans = scaler_dict[model_name]
    x = x_trans.fit_transform(x)

    return x, y

In [95]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV



def train_model(model_name):
    x, y = prep_data(model_name, X, Y)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=SEED)
    model = models_dict[model_name]
    grid_search = GridSearchCV(model, {}, scoring='roc_auc', cv=cv, n_jobs=-1)
    grid_search.fit(x, y)
    model = grid_search.best_estimator_
    print(f"Model: {model_name}, performance: {grid_search.best_score_}")


In [96]:
for model_name in reversed(models_dict.keys()):
    train_model(model_name)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Tr